In [1]:
import numpy as np
import tensorflow as tf
import re
import itertools
from collections import Counter
from collections import OrderedDict
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import collections
import math
import os
import random
from tempfile import gettempdir
import zipfile
import csv
import cPickle as pickle
print("11111")

11111


In [2]:
dictionary = OrderedDict()
with open("../embeddings/final_embeddings/vocab.txt", "r") as f:
    dictionary = pickle.load(f)
print("dictionary length: %d" % len(dictionary))
#print("dictionary: %s" % dictionary)
with tf.variable_scope("embeddings", reuse=tf.AUTO_REUSE):
    new_embeddings = tf.get_variable("embeddings", shape = [40000,200])#[vocabulary_size, embedding_size]
    norm = tf.sqrt(tf.reduce_sum(tf.square(new_embeddings), 1, keep_dims=True))
    restore = tf.train.Saver({"embeddings": new_embeddings})
    print (tf.shape(new_embeddings))
    with tf.Session() as sess:
        restore.restore(sess, "../embeddings/final_embeddings/model.ckpt")
        norm_embeddings = new_embeddings / norm
        print("new_embeddings: %s" %norm_embeddings.eval())
        final_embeddings = norm_embeddings.eval()
        print (tf.shape(norm_embeddings))

dictionary length: 40000
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Tensor("embeddings/Shape:0", shape=(2,), dtype=int32)
INFO:tensorflow:Restoring parameters from ../embeddings/final_embeddings/model.ckpt
new_embeddings: [[ 0.10012653 -0.0714228  -0.02289733 ...  0.12032924 -0.00925253
   0.10401342]
 [-0.04890159 -0.03326475  0.11969578 ...  0.04146247 -0.08580523
  -0.07447449]
 [-0.05911008  0.00246984 -0.01628511 ...  0.03341895  0.01533006
   0.10097369]
 ...
 [-0.10917597 -0.02065859  0.03950845 ... -0.04358016 -0.05217595
  -0.07356514]
 [-0.05918077  0.04198439  0.1179584  ... -0.07828391 -0.10345297
  -0.06059229]
 [ 0.08967408 -0.06848793  0.07969988 ... -0.09244967  0.06966498
   0.08792025]]
Tensor("embeddings/Shape_1:0", shape=(2,), dtype=int32)


In [3]:
class0_file = "../data/class-0.csv"
class1_file = "../data/class-1.csv"
# class2_file = "../data/class-2.csv"

testing_file = "../data/test.csv"

input_x = tf.placeholder(tf.int32, [None, None], name="input_x")
#y_labels = tf.placeholder(tf.float32, [None, 3], name="labels")
y_labels = tf.placeholder(tf.float32, [None, 1], name="labels")
dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

testing_x = tf.placeholder(tf.int32, [None, None], name="extension_lhs")


def load_data_and_labels():
    """
    Loads MR polarity data from files, splits the data into words and generates labels.
    Returns split sentences and labels.
    """

    
    
    # Load data from files
    x_class0 = list(open(class0_file, "r").readlines())
    print("size x_class0: "+ str(len(x_class0)))
    x_class1 = list(open(class1_file, "r").readlines())
    print("size x_class1: "+ str(len(x_class1)))
#     x_class2 = list(open(class2_file, "r").readlines())
#     print("size x_class2: "+ str(len(x_class2)))
    
#     x_testing = list(open(testing_file, "r").readlines())
#     x_testing = [x.lower() for x in x_testing]
   
    # Split by words
    #x_text = x_class0 + x_class1 + x_class2
    x_text = x_class0 + x_class1
    x_text = [x.lower() for x in x_text]
    print("leng(x_test): " + str(len(x_text)))
    
    
    #x_text = [clean_str(sent) for sent in x_text]
    
    # Generate labels
#     class_2_labels = [[0,0,1] for _ in x_class2]
#     class_1_labels = [[0,1,0] for _ in x_class1]
#     class_0_labels = [[1,0,0] for _ in x_class0]
    #class_4_labels = [[1,0,0,0] for _ in lhs_class4]
    
    class_1_labels = [[1] for _ in x_class1]
    class_0_labels = [[0] for _ in x_class0]
    
    #y = np.concatenate([class_0_labels, class_1_labels, class_2_labels], 0)
    
    y = np.concatenate([class_0_labels, class_1_labels],0)
#     return [x_text, y, x_testing]
    return [x_text, y]



def batch_iter(data, batch_size, num_epochs, shuffle=True):
    """
    Generates a batch iterator for a dataset.
    """
    data = np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]

In [4]:
import time
import datetime
from tensorflow.contrib import learn


# Parameters
# ==================================================

# Data loading params
dev_sample_percentage = .1

# Model Hyperparameters
embedding_dim=200 #128
filter_sizes= "1,2,3"
# num_filters =128
num_filters =16
dropout_keep_prob=1
learning_rate=.001
# learning_rate=.0008
num_classes=2
num_epochs=10


# Training parameters
batch_size=50




# Data Preparation
# ==================================================

# Load data
print("Loading data...")
# x_train, y, x_test= load_data_and_labels()
x_train, y = load_data_and_labels()
print("len(x_train):")
print(len(x_train))

# Build vocabulary  #my data 
x_text = x_train
max_phrase_length = max([len(x.split(" ")) for x in x_text])
vocab_processor_x = learn.preprocessing.VocabularyProcessor(max_phrase_length)
x = np.array(list(vocab_processor_x.fit_transform(x_text)))


# x_train = x[0:(len(x_train)),:]
# x_test = x[len(x_train):len(x),:]



## Extract word:id mapping from the object.
## Treat the id's as index into list and create a list of words in the ascending order of id's
## word with id i goes at index i of the list.
vocab_dict = vocab_processor_x.vocabulary_._mapping
sorted_vocab= sorted(vocab_dict.items(), key = lambda x : x[1])
vocabulary = list(list(zip(*sorted_vocab))[0])


# Randomly shuffle data (x_train, y)
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffle = x[shuffle_indices]
y_shuffle = y[shuffle_indices]
dev_sample_index = -1 * int(dev_sample_percentage * float(len(y)))
x_train, x_test = x_shuffle[:dev_sample_index], x_shuffle[dev_sample_index:]
y_train, y_test = y_shuffle[:dev_sample_index], y_shuffle[dev_sample_index:]

print(x_train)
print("len(x_train):")

print (len(x_train))

# no need
# class0_count = 0
# class1_count = 0
# class2_count = 0
# classes_count = np.argmax(y_train,1)
# for i in classes_count:
#     if i == 0:
#         class0_count = class0_count + 1
#     elif i == 1:
#         class1_count = class1_count + 1
#     elif i == 2:
#         class2_count = class2_count + 1
# total_count = len(y_train)
# no need 

del x, y

print("Vocabulary Size: {:d}".format(len(vocab_processor_x.vocabulary_)))
#print("RHS Vocabulary Size: {:d}".format(len(vocab_processor_rhs.vocabulary_)))
print("Train/Test split: {:d}/{:d}".format(len(y_train), len(x_test)))


Loading data...
size x_class0: 2607
size x_class1: 1186
leng(x_test): 3793
len(x_train):
3793
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
[[ 103   18    4 ...    0    0    0]
 [1233   53   51 ...    0    0    0]
 [  33  396   23 ...    0    0    0]
 ...
 [  51   44  240 ...    0    0    0]
 [  43  237   97 ...    0    0    0]
 [  51  240   44 ...    0    0    0]]
len(x_train):
3414
Vocabulary Size: 6756
Train/Test split: 3414/379


In [5]:
def convolutional_neural_network(input_x, sequence_length, vocab_size,
      embedding_size, filter_sizes, num_filters, initW): 
   # initW: sub embedding 
        #W = tf.Variable(
               #tf.random_uniform([len(vocab_processor_x.vocabulary_), 128], -1.0, 1.0),
               #name="W")
        #W.assign(initW)
        # Embedding layer
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
            embedded_chars = tf.nn.embedding_lookup(initW, input_x)
            embedded_chars_expanded = tf.expand_dims(embedded_chars, -1)
            
        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.device('/cpu:0'), tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                #print(embedded_chars_expanded.dtype)
                #print(W.dtype)
                #print(b.dtype)
                conv = tf.nn.conv2d(
                    embedded_chars_expanded,
                    W,
                    use_cudnn_on_gpu=False,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.tanh(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                #print(h.dtype)
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)
                
        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        h_pool = tf.concat(pooled_outputs, 3)
        h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])
        
        #dropout
        h_drop = tf.nn.dropout(h_pool_flat, dropout_keep_prob)
        
        return h_drop, num_filters_total
        



In [ ]:






# Training
# ==================================================

def train_neural_network(input_x):
    initW = np.random.uniform(-0.25,0.25,(len(vocab_processor_x.vocabulary_), embedding_dim)).astype(np.float32)
    # get the small embedding, subset of the final embedding
    for word in vocabulary:
        #print(word, dictionary.get(word,0))
        #print(final_embeddings[dictionary.get(word,0)])
        index = vocab_processor_x.vocabulary_.get(word)
        initW[index] = final_embeddings[dictionary.get(word,0)]
        #print('index', index)
    print(len(vocabulary))
   # print(vocabulary)
    
    # mnist cnn 
    vector_cnn,num_filters_total = convolutional_neural_network(input_x,sequence_length=x_train.shape[1],
            #num_classes=y_train.shape[1],
            vocab_size=len(vocab_processor_x.vocabulary_),
            embedding_size=embedding_dim,
            filter_sizes=list(map(int, filter_sizes.split(","))),
            num_filters=num_filters, initW = initW)
    
    
    dirW = tf.Variable(tf.random_normal([num_filters_total, num_classes]))
    bias = tf.Variable(tf.random_normal([num_classes]))
    prediction = tf.nn.sigmoid(tf.matmul(vector_cnn,dirW) + bias)
    
    #weighted loss
    #class_weights = tf.constant([[(class0_count)/total_count, class1_count/total_count, class2_count/total_count]])
    #print("total_count: " + str(total_count))
#     class_weights = tf.constant([[(class0_count)/total_count, class1_count/total_count]])
#     print(class_weights)
    #weight_per_label = tf.transpose(tf.matmul(y_labels, tf.transpose(class_weights)))
    xent = tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y_labels)
    
    cost = tf.reduce_mean(xent)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    
    with tf.name_scope('accuracy'):
        with tf.name_scope('correct_prediction'):
            pred = tf.argmax(prediction, 1)
            prob_max = tf.reduce_max(prediction, reduction_indices=[1])
            y = tf.argmax(y_labels,1)
            correct = tf.equal(pred, y)
        with tf.name_scope('accuracy'):
            accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
    hm_epochs = num_epochs
    with tf.Session() as sess:   
        tf.global_variables_initializer().run()
        for epoch in range(hm_epochs):
            batches = batch_iter(list(zip(x_train, y_train)), batch_size, hm_epochs)
            #print(epoch, hm_epochs)
            epoch_loss = 0
            for batch in batches:
                #print(batch, batches)
                x_batch, y_batch = zip(*batch)
                _,c, acc, = sess.run([optimizer,cost,accuracy], feed_dict={input_x:x_batch, y_labels: y_batch})
                epoch_loss += c
            
            
            # Record training set loss and accuracy for each epoch during training      
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss, 'training accuracy:',acc)
        print('Testing,accuracy:',accuracy.eval({input_x:x_test,y_labels:y_test}))

            #print('Testing Accuracy:',accuracy.eval({x:mnist.test.images,y:mnist.test.labels}))
            
#             if epoch == hm_epochs-1:
                
#                 #compute the probablities for the test set
#                 csvFile = open('test-set-predictions-cnn-model.csv', 'w')
#                 csvFile.write('ID,TITLE,TOPIC\n')
#                 probablities, predictions  = sess.run([prob_max,pred], feed_dict={input_x:x_test})
#                 print (len(probablities))
#                 print (probablities)
#                 index = 0
#                 for index in range(len(probablities)):
#                     headline = ""
#                     for i in range(len(x_test[index])):
#                         if(x_test[index][i] == 0):
#                             break
#                         else:
#                             headline = headline + vocabulary[x_test[index][i]] + " "
                    
#                     if predictions[index] == 0:
#                         p = "0"
# #                     if predictions[index] == 1:
# #                         p = "1"
#                     elif predictions[index] == 1:
#                         p = "1"
                    
#                     row = str(index) + "," + headline.strip()  + "," + str(predictions[index]) + "\n"
#                     csvFile.write(row)
#                 csvFile.close()


train_neural_network(input_x)

6756
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch 0 completed out of 10 loss: 298.3335052654147 training accuracy: 0.071428575
Epoch 1 completed out of 10 loss: 298.283020876348 training accuracy: 0.0
Epoch 2 completed out of 10 loss: 302.19557423889637 training accuracy: 0.0
Epoch 3 completed out of 10 loss: 329.3487078025937 training accuracy: 0.2857143
Epoch 4 completed out of 10 loss: 338.1139939725399 training accuracy: 0.14285715
Epoch 5 completed out of 10 loss: 341.5989620089531 training accuracy: 0.0
Epoch 6 completed out of 10 loss: 342.9463815689087 training accuracy: 0.14285715
Epoch 7 completed out of 10 loss: 343.65974570810795 training accuracy: 0.71428573
Epoch 8 completed out of 10 loss: 298.91296216100454 training accuracy: 0.64285713
